In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader

from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.documents import Document
import os
import ollama
import chromadb

client = chromadb.Client()
collection = client.create_collection(name="docs")

In [2]:
def load_documents(folder_path: str) -> List[Document]:
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        elif filename.endswith('.md'):
            loader = UnstructuredMarkdownLoader(file_path)
        else:
            print(f"unsupported file type: {filename}")
            continue
        documents.extend(loader.load())
    return documents

MAX_LENGTH = 1000  

def split_documents(documents: List[Document]) -> List[Document]:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=200,
        chunk_overlap=75,
        length_function=len
    )

    final_docs = []
    for doc in documents:
        content = doc.page_content

        if len(content) > MAX_LENGTH:
            chunks = text_splitter.split_documents([doc])
            print(f"Splitting document: {len(content)} chars → {len(chunks)} chunks")
            final_docs.extend(chunks)
        else:
            final_docs.append(doc)

    return final_docs


def embednstore(splits, collection):
    for i, doc in enumerate(splits):
        print(f"Document {i}:", doc.page_content[:200])  # Preview first 200 chars

        text = doc.page_content
        response = ollama.embed(model="mxbai-embed-large", input=text)
        embedding = response["embeddings"][0]

        collection.add(
            ids=[str(i)],
            embeddings=[embedding],
            documents=[text]
        )
        print(f"Embedding {i} length: {len(embedding)} | Preview: {embedding[:5]}")
    

    
documents = load_documents("/home/ruta/irishep/hint_files")
print(f"loaded {len(documents)} documents from the folder")

splits = split_documents(documents)
print(f"split the documents into {len(splits)} chunks")\
    
embednstore(splits, collection)
print(f"Stored {len(splits)} embedded chunks in ChromaDB.")


loaded 2 documents from the folder
Splitting document: 1286 chars → 10 chunks
Splitting document: 2434 chars → 18 chunks
split the documents into 28 chunks
Document 0: 0. LIBRARIES

import uproot import awkward as ak import matplotlib.pyplot as plt

1. BASIC FILE OPERATIONS
Embedding 0 length: 1024 | Preview: [0.051768765, 0.0059163184, 0.030181054, -0.058384478, 0.01449202]
Document 1: 1. BASIC FILE OPERATIONS

file = uproot.open("file.root") print(file.keys()) print(file.classnames()) print(file["Events"].num_entries)

2. LOADING DATA
Embedding 1 length: 1024 | Preview: [0.023202023, 0.0084609175, 0.01455583, -0.051943798, 0.005220387]
Document 2: 2. LOADING DATA

tree = file["Events"] branches = tree.arrays() selected = tree.arrays(["Muon_pt", "Muon_eta"])

3. WORKING WITH JAGGED ARRAYS
Embedding 2 length: 1024 | Preview: [0.0010778526, -0.002176215, 0.0013118689, -0.02484658, -0.014950862]
Document 3: 3. WORKING WITH JAGGED ARRAYS

muon_pt = branches["Muon_pt"] print(muon_pt[0].tol

In [4]:
input = "how would you plot the muon pT of the file '4AAF4AB2-171D-F54C-8FE3-0D709B049A8A.root'?"

resp = ollama.embed(model="mxbai-embed-large", input=input)
query_embedding = resp["embeddings"][0] 

results = collection.query(
    query_embeddings=[query_embedding],
    n_results=5
)

data = results['documents'][0][0]

print("Top retrieved document:", data)

output = ollama.generate(
    model="llama3",
    prompt = f"""You are a helpful assistant with access to these CMS specific hint files with python code snippets: {data}
            Only use the above data to answer the following question, without hallucinating or making up your own statements: {input},
            The expected output is a python code snippet that can be run.
            If the answer is not in the provided data, say "I don't know based on the available information,
            PLEASE MAKE SURE TO IMPORT NECESSARY LIBRARIES."
        """,    
)
#try with no chunking
#no chuunking unless very large hints
#characterize how far the model can go with the data
#outline with goals
#in chunks basic workflow cuz this llama is kinda dumb


print("RESPONSE: \n " +   output['response'])
client.delete_collection("docs")
collection = client.create_collection(name="docs")


Top retrieved document: 0. LIBRARIES

import uproot import awkward as ak import matplotlib.pyplot as plt

1. BASIC FILE OPERATIONS

file = uproot.open("file.root") print(file.keys()) print(file.classnames()) print(file["Events"].num_entries)

2. LOADING DATA

tree = file["Events"] branches = tree.arrays() selected = tree.arrays(["Muon_pt", "Muon_eta"])

3. WORKING WITH JAGGED ARRAYS

muon_pt = branches["Muon_pt"] print(muon_pt[0].tolist()) print(ak.num(muon_pt)) print(ak.flatten(muon_pt)) first_muon_pt = ak.firsts(muon_pt) print(first_muon_pt)

4. SELECTIONS AND FILTERING

good_pt = branches["Muon_pt"] > 20 good_muons = branches["Muon_pt"][good_pt]

mask = (branches["Muon_pt"] > 20) & (abs(branches["Muon_eta"]) < 2.4) events_with_good_muons = ak.any(mask, axis=1) filtered_events = branches[events_with_good_muons]

5. PLOTTING

plt.hist(ak.flatten(branches["Muon_pt"]), bins=50, range=(0, 100)) plt.xlabel("Muon pT [GeV]") plt.ylabel("Counts") plt.title("Muon Transverse Momentum") plt.sho